In [1]:
# -*- coding: utf-8 -*-
import re

LHan = [[0x2E80, 0x2E99],    # Han # So  [26] CJK RADICAL REPEAT, CJK RADICAL RAP
        [0x2E9B, 0x2EF3],    # Han # So  [89] CJK RADICAL CHOKE, CJK RADICAL C-SIMPLIFIED TURTLE
        [0x2F00, 0x2FD5],    # Han # So [214] KANGXI RADICAL ONE, KANGXI RADICAL FLUTE
        0x3005,              # Han # Lm       IDEOGRAPHIC ITERATION MARK
        0x3007,              # Han # Nl       IDEOGRAPHIC NUMBER ZERO
        [0x3021, 0x3029],    # Han # Nl   [9] HANGZHOU NUMERAL ONE, HANGZHOU NUMERAL NINE
        [0x3038, 0x303A],    # Han # Nl   [3] HANGZHOU NUMERAL TEN, HANGZHOU NUMERAL THIRTY
        0x303B,              # Han # Lm       VERTICAL IDEOGRAPHIC ITERATION MARK
        [0x3400, 0x4DB5],    # Han # Lo [6582] CJK UNIFIED IDEOGRAPH-3400, CJK UNIFIED IDEOGRAPH-4DB5
        [0x4E00, 0x9FC3],    # Han # Lo [20932] CJK UNIFIED IDEOGRAPH-4E00, CJK UNIFIED IDEOGRAPH-9FC3
        [0xF900, 0xFA2D],    # Han # Lo [302] CJK COMPATIBILITY IDEOGRAPH-F900, CJK COMPATIBILITY IDEOGRAPH-FA2D
        [0xFA30, 0xFA6A],    # Han # Lo  [59] CJK COMPATIBILITY IDEOGRAPH-FA30, CJK COMPATIBILITY IDEOGRAPH-FA6A
        [0xFA70, 0xFAD9],    # Han # Lo [106] CJK COMPATIBILITY IDEOGRAPH-FA70, CJK COMPATIBILITY IDEOGRAPH-FAD9
        [0x20000, 0x2A6D6],  # Han # Lo [42711] CJK UNIFIED IDEOGRAPH-20000, CJK UNIFIED IDEOGRAPH-2A6D6
        [0x2F800, 0x2FA1D]]  # Han # Lo [542] CJK COMPATIBILITY IDEOGRAPH-2F800, CJK COMPATIBILITY IDEOGRAPH-2FA1D

def build_re():
    L = []
    for i in LHan:
        if isinstance(i, list):
            f, t = i
            try: 
                f = chr(f)
                t = chr(t)
                L.append('%s-%s' % (f, t))
            except: 
                pass # A narrow python build, so can't use chars > 65535 without surrogate pairs!

        else:
            try:
                L.append(chr(i))
            except:
                pass

    RE = '[%s]' % ''.join(L)
    print('RE:', RE.encode('utf-8'))
    return re.compile(RE, re.UNICODE)

RE = build_re()

RE: b'[\xe2\xba\x80-\xe2\xba\x99\xe2\xba\x9b-\xe2\xbb\xb3\xe2\xbc\x80-\xe2\xbf\x95\xe3\x80\x85\xe3\x80\x87\xe3\x80\xa1-\xe3\x80\xa9\xe3\x80\xb8-\xe3\x80\xba\xe3\x80\xbb\xe3\x90\x80-\xe4\xb6\xb5\xe4\xb8\x80-\xe9\xbf\x83\xef\xa4\x80-\xef\xa8\xad\xef\xa8\xb0-\xef\xa9\xaa\xef\xa9\xb0-\xef\xab\x99\xf0\xa0\x80\x80-\xf0\xaa\x9b\x96\xf0\xaf\xa0\x80-\xf0\xaf\xa8\x9d]'


In [2]:
import sqlite3
from nltk import FreqDist

In [3]:
db = sqlite3.connect('preach_my_gospel_zh.db')
cur = db.cursor()

In [15]:
# "CREATE TABLE corpus (url TEXT PRIMARY KEY, type TEXT, html TEXT, string TEXT)"
sql = 'SELECT string, url FROM corpus'
cur.execute(sql)
data = cur.fetchall()

In [5]:
zi = RE.findall('\n'.join([x[0] for x in data]))
fdist = FreqDist(zi)
len(zi)

274269

In [51]:
fdist2 = FreqDist()
learned = 400
print(str(int(sum([x[1] for x in fdist.most_common(learned)])/len(zi)*100))+"%")
most_common = [x[0] for x in fdist.most_common(learned)]
for text, url in data:
    characters = set(RE.findall(text))
    for character in characters:
        if character not in most_common:
            break
    else:
        if len(text) > 3:
            for a in set(RE.findall(text)):
                fdist2[a] += 1

86%


In [59]:
first_learned = [x[0] for x in fdist2.most_common(64)]
first_learned.reverse()
print('\t'.join([x for x in first_learned if (first_learned.index(x)-7)%8 == 0]))
print('\t'.join([x for x in first_learned if (first_learned.index(x)-6)%8 == 0]))
print('\t'.join([x for x in first_learned if (first_learned.index(x)-5)%8 == 0]))
print('\t'.join([x for x in first_learned if (first_learned.index(x)-4)%8 == 0]))
print('\t'.join([x for x in first_learned if (first_learned.index(x)-3)%8 == 0]))
print('\t'.join([x for x in first_learned if (first_learned.index(x)-2)%8 == 0]))
print('\t'.join([x for x in first_learned if (first_learned.index(x)-1)%8 == 0]))
print('\t'.join([x for x in first_learned if first_learned.index(x)%8 == 0]))
first_learned.reverse()

祈	那	說	中	做	有	給	的
如	話	他	為	信	道	不	你
誡	力	而	會	必	著	所	我
果	量	麼	看	教	靈	一	事
愛	能	心	命	藉	是	在	要
樣	實	且	奉	神	這	到	就
或	啊	都	父	切	求	以	們
像	向	去	情	人	只	並	知


In [72]:
print(str(int(sum([fdist[x[0]] for x in fdist2.most_common(learned)])/len(zi)*100))+"%")
most_common = [x[0] for x in fdist.most_common(learned)]
for text, url in data:
    score = 0
    characters = set(RE.findall(text))
    for character in characters:
        if character not in most_common:
            break
    else:
        if len(text) > 3:
            for a in set(RE.findall(text)):
                score += fdist2[a]
            text = re.sub(r'[ａｂｃ0-9]+','', text)
            print(score/len(set(RE.findall(text))),'\t','[',text,'](',url,')',sep='')

58%
4.0476190476190474	[看啊，你還不了解；你以為不用思考，只要向我祈求，我就會給你。](https://www.lds.org/scriptures/dc-testament/dc/9.7?lang=zho#p6)
5.647058823529412	[如果你愛我，就要事奉我，並遵守我所有的誡命。](https://www.lds.org/scriptures/dc-testament/dc/42.29?lang=zho#p28)
4.833333333333333	[所以我希望你們像我一樣完全，或像你們在天上的父一樣完全。](https://www.lds.org/scriptures/bofm/3-ne/12.48?lang=zho#p47)
4.333333333333333	[我對他說：我知道祂愛祂的兒女；不過，我不明白所有事情的意義。](https://www.lds.org/scriptures/bofm/1-ne/11.17?lang=zho#p16)
4.176470588235294	[而我被靈引領著，事先並不知道該做什麼。](https://www.lds.org/scriptures/bofm/1-ne/4.6?lang=zho#p5)
4.88	[靈必藉著信心的祈禱而給你們；你們如果沒有接受到靈，就不要教導。](https://www.lds.org/scriptures/dc-testament/dc/42.14?lang=zho#p13)
4.478260869565218	[所以，你做的一切事都要奉子的名去做，你要悔改，並且永永遠遠奉子的名呼求神。](https://www.lds.org/scriptures/pgp/moses/5.8?lang=zho#p7)
3.5869565217391304	[我，尼腓，不能將在我人民中教導的所有的事都寫下來；而且，我寫的不像說的那麼有力；因為人藉著聖靈的力量講話時，聖靈的力量就將他的話帶到人類兒女心上。](https://www.lds.org/scriptures/bofm/2-ne/33.1?lang=zho#p0)
4.833333333333333	[人在無知中得救是不可能的。](https://www.lds.org/scriptures/dc-testam